# Reducers
Reducers are key to understanding how updates from nodes are applied to the State. Each key in the State has its own independent reducer function. If no reducer function is explicitly specified then it is assumed that all updates to that key should override it. There are a few different types of reducers, starting with the default type of reducer:

## Default Reducer

### Example 1: Overwrite or Update

In this example, no reducer functions are specified for any key. Let's assume the input to the graph is `{"foo": 1, "bar": ["hi"]}`.  
Let's then assume the first Node returns `{"foo": 2}`.  
This is treated as an update to the state. Notice that the Node does not need to return the whole State schema - just an update.  
After applying this update, the State would then be `{"foo": 2, "bar": ["hi"]}`.  
If the second node returns {"bar": ["bye"]} then the State would then be `{"foo": 2, "bar": ["bye"]}`

In [ ]:
from typing_extensions import TypedDict
from langchain_core.tools import tool

class State(TypedDict):
    foo: int
    bar: list[str]

### Example 2: Reducer function

In this example, we've used the Annotated type to specify a reducer function (operator.add) for the second key (bar).   
Note that the first key remains unchanged.  
Let's assume the input to the graph is {"foo": 1, "bar": ["hi"]}.  
Let's then assume the first Node returns {"foo": 2}.  
This is treated as an update to the state.  
Notice that the Node does not need to return the whole State schema - just an update.  
After applying this update, the State would then be {"foo": 2, "bar": ["hi"]}.  
If the second node returns {"bar": ["bye"]} then the State would then be {"foo": 2, "bar": ["hi", "bye"]}.  
Notice here that the bar key is updated by adding the two lists together.  

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add]

> NOTE:  If you don't specify a reducer, every state update will overwrite the list of messages with the most recently provided value. If you wanted to simply append messages to the existing list, you could use operator.add as a reducer.

In [26]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add_messages]


A reducer that can keep track of message IDs and overwrite existing messages, if updated. 
(Allows to append or update messages in the list)

`add_messages` : For brand new messages, it will simply append to existing list, but it will also handle the updates for existing messages correctly.

In [30]:
s = State(foo=1, bar=["a", "b"])
type(s), s

(dict, {'foo': 1, 'bar': ['a', 'b']})

In [33]:
s['bar']

['a', 'b']

In [32]:
from typing import Callable, List, Any
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages


class GraphState(dict):
    """
    Manages a dictionary-like structure with automatic message summarization
    when the message list exceeds a certain length.
    """
    def __init__(self, limit: int, summarize_func: Callable[[List[HumanMessage]], HumanMessage]):
        super().__init__()
        self["messages"]: List[HumanMessage] = []
        self["limit"] = limit
        self.summarize_func = summarize_func

    def add_message(self, message: HumanMessage) -> None:
        """
        Adds a message to the list. If the total number of messages exceeds the limit, 
        triggers summarization.
        """
        self["messages"].append(message)
        if len(self["messages"]) > self["limit"]:
            self._summarize()

    def _summarize(self) -> None:
        """
        Summarizes the messages and reduces the list to a summary and the last few messages.
        """
        # Messages to summarize (all except the last few)
        messages_to_summarize = self["messages"][:-self["limit"]]
        # Create a summary message using the provided summarization function
        summary_message = self.summarize_func(messages_to_summarize)
        # Keep the last `limit` messages for context and prepend the summary
        self["messages"] = [summary_message] + self["messages"][-self["limit"]:]

    # Overriding __getitem__ and __setitem__ for dictionary-like behavior
    def __getitem__(self, key: str) -> Any:
        return super().__getitem__(key)

    def __setitem__(self, key: str, value: Any) -> None:
        super().__setitem__(key, value)


# Example summarization function
def example_summarize_func(messages: List[HumanMessage]) -> HumanMessage:
    """
    Summarizes a list of messages by concatenating their content into a single message.
    """
    summary_content = " ".join([msg.content for msg in messages])  # Combine all message contents
    return HumanMessage(content=f"Summary: {summary_content}")


# Example usage
graph_state = GraphState(limit=5, summarize_func=example_summarize_func)
type(graph_state)


__main__.GraphState

In [36]:
graph_state

{'messages': [], 'limit': 5}

In [37]:

# Add example messages
for i in range(1, 11):  # Adding 10 messages
    graph_state.add_message(HumanMessage(content=f"Message {i}"))
    print(f"After adding Message {i}: {[msg.content for msg in graph_state['messages']]}")

After adding Message 1: ['Message 1']
After adding Message 2: ['Message 1', 'Message 2']
After adding Message 3: ['Message 1', 'Message 2', 'Message 3']
After adding Message 4: ['Message 1', 'Message 2', 'Message 3', 'Message 4']
After adding Message 5: ['Message 1', 'Message 2', 'Message 3', 'Message 4', 'Message 5']
After adding Message 6: ['Summary: Message 1', 'Message 2', 'Message 3', 'Message 4', 'Message 5', 'Message 6']
After adding Message 7: ['Summary: Summary: Message 1 Message 2', 'Message 3', 'Message 4', 'Message 5', 'Message 6', 'Message 7']
After adding Message 8: ['Summary: Summary: Summary: Message 1 Message 2 Message 3', 'Message 4', 'Message 5', 'Message 6', 'Message 7', 'Message 8']
After adding Message 9: ['Summary: Summary: Summary: Summary: Message 1 Message 2 Message 3 Message 4', 'Message 5', 'Message 6', 'Message 7', 'Message 8', 'Message 9']
After adding Message 10: ['Summary: Summary: Summary: Summary: Summary: Message 1 Message 2 Message 3 Message 4 Messag